<a href="https://colab.research.google.com/github/Nicompag/ISYS5002_portfolio/blob/main/ISYS5002_Assignment_20201867_Nico_Engler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment ISYS5002

#####Curtin University
#####Semester 2 - 2021
#####Nico Engler (20201867)

##Steps

1. Get input: Prompt the use for a ticker. Make sure the ticker is valid and is an Australian ticker. It does not make sense to calculate the tax with the Australian tax table for incomes in other countries and currencies.

2. Get the data: Go to Yahoo Finance and get the executive table. Find a way to get some form of raw data from the information on the website.

3. Extract collection: Extract all neccessary information (at least name and salary) from that raw information. Make sure the functions have some form of error handling. Also, sort the data that you can manipulate it with pandas.

4. Store data: Prompt the user for the favoured data storaged. Give choice for csv file, SQLite Database or both. 

5. Putting it together: Combined all functions so that the program runs at once. Make sure there is sufficient input validation and error handling. Also, describe each function for other users. 

##Import Modules

Add here all modules you are going to use for the program

In [ ]:
#First I tried to get the data via the yfinance module
#!pip install yfinance
#import yfinance as yfin
#yfin.pdr_override()

!pip install kora -q
import requests
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd


#1. Get Input

First input validation idea

In [ ]:
ticker = input('''Please enter the company's stock market ticker symbol: ''')


if ticker.isdigit() == True:
print('Please enter a valid stock market ticker symbol!')



More effective input validation

In [ ]:
def get_input():
  while True:
    ticker = input('''Please enter an Australian stock market ticker symbol: ''')
    if ticker[-3:] == '.ax' or ticker[-3:] == '.AX':
      ticker = ticker
      break
    else:
      print('Please enter an Australian ticker.')
  return ticker

ticker = get_input()
ticker

#2. Get Data

Establish the session using the kora selenium module.

In [ ]:
url = 'https://au.finance.yahoo.com/'
wd.get(url)

Work out the pattern of the desired URL

In [ ]:
# Work out URL pattern
template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'

url = template.format(ticker1 = ticker, ticker2 = ticker)
wd.get(url)

Create a function with the pattern

In [ ]:
def get_url(ticker):
  template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'
  url = template.format(ticker1 = ticker, ticker2 = ticker)

  return url

Using the BeautifulSoup module to get the raw data

In [ ]:
soup = BeautifulSoup(wd.page_source, 'html.parser')

#3. Extract Collection

Find the right information in the raw data

In [ ]:
table = soup.find_all('tr', {'class': 'C($primaryColor) BdB Bdc($seperatorColor) H(36px)'})
table[4].td.text

Find the name from the sorted raw data

In [ ]:
def extract_name(item):
  name = item.td.text
  return name

Find the salary from the sorted raw data. Understand pattern to convert salary in string format into float.

In [ ]:
item = table[0]
salary = item.find('td', 'Ta(end)')
salary = salary.text

salary[-1]

Build function to extract salary and convert into float

In [ ]:
def extract_salary(item):
  try:
    salary = item.find('td', 'Ta(end)')
    salary = salary.text 
    if salary[-1] == "M":
      salary = float(salary[:-1])*1000000
    else:
      salary = float(salary[:-1])*1000
  except ValueError:
    salary = 'N/A'
  return salary

Test salary for several executives

In [ ]:
extract_salary(table[6])

Find the title from sorted raw data and build function to extract the title

In [ ]:
def extract_title(item):
  title = item.find('td', 'Ta(start) W(45%)')
  title = title.text
  return title

Calculate the tax of a given salary. If not sure how to handle not available salaries, create error handling so the program works in any case.

In [ ]:
def calculate_tax(income):
  try:
    if income <= 18200:
      tax_payable = 0
    elif 18200 < income <= 45000:
      tax_payable = (income - 18200) * 0.19
    elif 45000 < income <= 120000:
      tax_payable = 5092 + 0.325 * (income - 45000)
    elif 120000 < income <= 180000:
      tax_payable = 29467 + 0.37 * (income - 120000)
    elif income > 180000:
      tax_payable = 51667 + 0.45 * (income - 180000)
  except TypeError:
      tax_payable = 'N/A'
  return tax_payable

Create a function to combine all findings into a dictionary for further use.

In [ ]:
def extract_record(item):
  return {
      'Name': extract_name(item),
      'Title': extract_title(item),
      'Salary': extract_salary(item),
      'Expected income tax': calculate_tax(extract_salary(item))
  }

Find a way to extract the information of each executive in the sorted raw data

In [ ]:
for i in table:
  print(extract_name(i))

Combine all executive dictionaries into a list.

In [ ]:
records = []
for i in table:
  records.append(extract_record(i))

Convert that list into a dataframe in pandas to visualise information and make it available for further use.

In [ ]:
df = pd.DataFrame.from_records(records)
df

#4. Store Data

Find a way to store data in the favoured data format.

In [11]:
#store data
place = input('''Where do you want to store the data? 
Press 1 for cvs file,
Press 2 for data,
Press 3 for both.
''')

place

Where do you want to store the data? 
Press 1 for cvs file,
Press 2 for data,
Press 3 for both.
1


'1'

Create function for saving the data.

#5. Putting it together

In [14]:
!pip install kora -q
'''load packages'''
import requests
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd

def get_url(ticker):
  ''' 
  This function gets the url from Yahoo Finance.
  '''
  template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'
  url = template.format(ticker1 = ticker, ticker2 = ticker)
  return url

def extract_name(item):
  '''
  This function extracts the name of the executive.
  '''
  name = item.td.text
  return name

def extract_title(item):
  '''
  This function gets the title or role from the respective executive.
  '''
  title = item.find('td', 'Ta(start) W(45%)')
  title = title.text
  return title

def extract_salary(item):
  '''
  This function extracts the salary of the executive. It also converts the salary from a string into a number (float). 
  If no salary is given ("N/A"), the salary is assumed to be 0.
  '''
  try:
    salary = item.find('td', 'Ta(end)')
    salary = salary.text
    if salary[-1] == "M":
      salary = float(salary[:-1])*1000000
    else:
      salary = float(salary[:-1])*1000 
  except ValueError:
    salary = 0
  return salary

def calculate_tax(income):
  '''
  This function calculates the tax payable from a given salary. If the salary is not a number, the income tax is assumed to be 0.
  '''
  try:
    if income <= 18200:
      tax_payable = 0
    elif 18200 < income <= 45000:
      tax_payable = (income - 18200) * 0.19
    elif 45000 < income <= 120000:
      tax_payable = 5092 + 0.325 * (income - 45000)
    elif 120000 < income <= 180000:
      tax_payable = 29467 + 0.37 * (income - 120000)
    elif income > 180000:
      tax_payable = 51667 + 0.45 * (income - 180000)
  except TypeError:
      tax_payable = 0
  return tax_payable

def extract_record(item):
  '''
  This functions combines the name, title, salary and expected income tax from each executive in the table.
  The output is a list made of dictionaries containing the information for each executive.
  '''
  return {
      'Name': extract_name(item),
      'Title': extract_title(item),
      'Salary': extract_salary(item),
      'Expected income tax': calculate_tax(extract_salary(item))
  }

def ticker_validation(dataframe):
  '''
  This functions validates whether the given ticker is a valid ticker. 
  If there are no information on Yahoo Finance under the given ticker, the user will be prompt to enter a new ticker.
  '''
  while True:
    if dataframe.empty:
      print('No information under this ticker. Please enter a valid ticker.')
      ticker = get_input()
      url = get_url(ticker)
      wd.get(url)
      soup = BeautifulSoup(wd.page_source, 'html.parser')
      records = []
      table = soup.find_all('tr', {'class': 'C($primaryColor) BdB Bdc($seperatorColor) H(36px)'})
      for i in table:
        records.append(extract_record(i))   
      dataframe = pd.DataFrame.from_records(records)
    else: 
      break
  return dataframe

def get_input():
  '''
  This function makes sure that the entered ticker belongs to an Australian company that is listed on the Australian Stock Exchange.
  If the ticker is not at traded at the ASX, the user will be prompt to enter a new ticker.
  '''
  while True:
    ticker = input('''Please enter an Australian stock market ticker symbol: ''')
    if ticker[-3:] == '.ax' or ticker[-3:] == '.AX':
      ticker = ticker
      break
    else:
      print('Please enter an Australian ticker.')
  return ticker

In [13]:
ticker = get_input()
url = get_url(ticker)
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'html.parser')
records = []
table = soup.find_all('tr', {'class': 'C($primaryColor) BdB Bdc($seperatorColor) H(36px)'})

for i in table:
  records.append(extract_record(i))

df = pd.DataFrame.from_records(records)
df = ticker_validation(df)
df

Please enter an Australian stock market ticker symbol: nab.ax


,Name,Title,Salary,Expected income tax
0,"Mr. Ross Maxwell McEwan CBE, B.B.S.","Group CEO, MD & Director",2540000.0,1113667.0
1,"Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons)...",Group Chief Financial Officer,1130000.0,479167.0
2,Mr. Patrick F. Wright,Group Exec. of Technology & Enterprise Operations,1720000.0,744667.0
3,Ms. Sharon J. Cook,Group Exec. of Legal & Commercial Services,912980.0,381508.0
4,Ms. Susan Ferrier,Group Exec. of People & Culture,929270.0,388838.5
5,Ms. Angela Mentis B.Bus,"Group Chief Digital, Data & Analytics Officer",1620000.0,699667.0
6,Mr. Shaun Dooley,Chief Risk Officer,1050000.0,443167.0
7,Mr. Andrew Irvine,Chief Exec. of Bus. Private Banking Division,333270.0,120638.5
8,Mr. Lesile D. Matheson,Group Chief Operating Officer,0.0,0.0
9,Ms. Sally Mihell,Head of Investor Relations,0.0,0.0
